<a href="https://colab.research.google.com/github/KCachel/FairRankTune/blob/main/examples/6_fairranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

We need to install [FairRankTune](https://https://github.com/KCachel/FairRankTune).

In [1]:
!pip install FairRankTune

We need to import FairRankTune along with some other packages.

In [2]:
import FairRankTune as frt
import numpy as np
import pandas as pd
from FairRankTune import RankTune, Metrics, Rankers

# Fair Ranking Algorithms
`FairRankTune` provides [fair ranking algorithms](https://kcachel.github.io/FairRankTune/Rankers/) in the `Rankers` module. These fair ranking algorithms can be used to re-rank a given ranking with the objective of making the resulting ranking fair.



## Epsilon-Greedy Re-Ranker
Epsilon-Greedy takes as input a ranking and repeatedly swaps pairs of items so that each item has probability $\epsilon$ (`epsilon`) of swapping with a random item below it. It does not require a specific notion of fairness or prior knowledge of group distributions. It does use random swapping, thus it is recommended to set a random seed for reproducability. To learn more see [Feng et al.](https://doi.org/10.1609/aaai.v36i11.21445) where it was introduced to improve group fairness.

Note that `epsilon` must be between $[0,1]$ and a `seed` is passed for reproducability.

Below we show Epsilon Greedy re-ranking a biased ranking produced by RankTune.

In [3]:
#Generate a biased (phi = 0) ranking of 1000 items, with two groups of 100 and 900 items each.
seed = 2 #For reproducability
group_proportions = np.asarray([.1, .9]) #Array of group proportions
num_items = 1000 #1000 items to be in the generated ranking
phi = 0 #Biased ranking
r_cnt = 1 #Generate 1 ranking
ranking_df, item_group_dict, scores_df = frt.RankTune.ScoredGenFromGroups(group_proportions,  num_items, phi, r_cnt, 'uniform', seed)

#Calculate EXP with a MinMaxRatio
EXP_minmax, avg_exposures_minmax = frt.Metrics.EXP(ranking_df, item_group_dict, 'MinMaxRatio')
print("EXP before Epsilon-Greedy: ", EXP_minmax, "avg_exposures before Epsilon-Greedy: ", avg_exposures_minmax)


#Rerank using Epsilon-Greedy

seed = 2 #For reproducability
epsilon = .6
reranking_df, item_group_d, reranking_scores = frt.Rankers.EPSILONGREEDY(ranking_df, item_group_dict, scores_df, epsilon, seed)

#Calculate EXP with a MinMaxRatio post Epsilon-Greedy
EXP, avg_exposures= frt.Metrics.EXP(reranking_df, item_group_d, 'MinMaxRatio')
print("EXP after Epsilon-Greedy: ", EXP, "avg_exposures after Epsilon-Greedy: ", avg_exposures)

EXP before Epsilon-Greedy:  0.5420744267551784 avg_exposures before Epsilon-Greedy:  {0: 0.2093867087428094, 1: 0.11350318011191189}
EXP after Epsilon-Greedy:  0.7689042373241246 avg_exposures after Epsilon-Greedy:  {0: 0.15541589156986096, 1: 0.1194999375755728}


We observe that the EXP score has increased (0.54 to 0.76) indicating the groups receive more comporable amounts of average exposure.

## DetConstSort Re-Ranker

DetConstSort takes as input a given ranking, and re-ranks items in it to create a top-k fair ranking. Fairness is achieved by setting the `distribution` dictionary.  In `distribution` the keys are group identifiers and the value is the desired group proportion. For any particular position k and for any group `g`, DetConstSort ensures that group occurs $\lfloor$ `distribution[g]` $*k \rfloor$ in the resulting ranking. DetConstSort algorithm also tries improve the utility of the ranking by ensuring that items with better scores are placed higher in the ranking so long as the ranking satisfies the feasibility criteria. To learn more see [Geyik et al.](https://dl.acm.org/doi/10.1145/3292500.3330691).

Below we show DetConstSort re-ranking the same biased ranking as above from RankTune.

In [4]:
#Generate a biased (phi = 0) ranking of 1000 items, with two groups of 100 and 900 items each.
seed = 2 #For reproducability
group_proportions = np.asarray([.1, .9]) #Array of group proportions
num_items = 1000 #1000 items to be in the generated ranking
phi = 0 #Biased ranking
r_cnt = 1 #Generate 1 ranking
ranking_df, item_group_dict, scores_df = frt.RankTune.ScoredGenFromGroups(group_proportions,  num_items, phi, r_cnt, 'uniform', seed)

#Calculate EXP with a MinMaxRatio
EXP_minmax, avg_exposures_minmax = frt.Metrics.EXP(ranking_df, item_group_dict, 'MinMaxRatio')
print("EXP before DetConstSort: ", EXP_minmax, "avg_exposures before DetConstSort: ", avg_exposures_minmax)

#Rerank using DetConstSort
distribution = dict(zip([0, 1], [.1, .9])) #set distribution for statistical parity
k = 800 #only ranking 800 items of the provided 1000
reranking_df, item_group_d, reranking_scores = frt.Rankers.DETCONSTSORT(ranking_df, item_group_dict, scores_df, distribution, k)

#Calculate EXP with a MinMaxRatio post DetConstSort
EXP, avg_exposures= frt.Metrics.EXP(reranking_df, item_group_d, 'MinMaxRatio')
print("EXP after DetConstSort: ", EXP, "avg_exposures after DetConstSort: ", avg_exposures)


EXP before DetConstSort:  0.5420744267551784 avg_exposures before DetConstSort:  {0: 0.2093867087428094, 1: 0.11350318011191189}
EXP after DetConstSort:  0.8892953016490737 avg_exposures after DetConstSort:  {0: 0.14259348136033306, 1: 0.12680771301952895}


We observe that the EXP score has increased (0.54 to 0.88) indicating the groups receive more comporable amounts of average exposure in this new top-800 ranking.